In [1]:
from bs4 import BeautifulSoup
import selenium
import random
from time import sleep
import pandas as pd
import logging 
from datetime import datetime
import pickle
import asyncio
import aiohttp
import nest_asyncio
nest_asyncio.apply()

In [2]:
logger = logging.getLogger()
dstamp = datetime.now().strftime('%m_%d')
fhandler = logging.FileHandler(filename= dstamp + 'session.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

#  Constants

## To Add Semaphores

##  Add try catch 

In [3]:
#needed so we won't be blocked
headersff = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0'} 

In [4]:
async def crWriteContent(n, content):
    filename = f'dl/async_{n}.html'
    with open(filename,'wb') as f:
        logging.info(f'Started writing to {filename}')
        f.write(content)
        logging.info(f'Finished writing to {filename}')

In [5]:
async def crRequestUrl(url):
    logging.info(f'Started downloading {url}')
    async with aiohttp.ClientSession(headers = headersff) as session:
        async with session.get(url) as resp:
            #sleep(20*(random.random()))
            waitTime = 3*(random.random())
            await asyncio.sleep(waitTime)
            content = await resp.read()
            logging.info(f'Downloaded {url}')
        return content 


In [6]:
# tasks order
async def crtaskRequestAndWrite(n,url):
    content = await crRequestUrl(url)
    await  crWriteContent(n,content)

In [7]:
#generation of task list, ie number of iter of the called taskTasks orders calls await functions 
async def crMain(urllist):
    tasklist = []
    for n, url in enumerate(urllist):
        tasklist.append(crtaskRequestAndWrite(n,url))
    await asyncio.wait(tasklist)

In [8]:
#Main loop
if __name__ == '__main__':
    file = open('hotelUlrs.pickle','rb')
    hotelUrls= pickle.load(file)
    #needed so we won't be blocked
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:73.0) Gecko/20100101 Firefox/73.0'} 
    file.close()
    loop = asyncio.get_event_loop()
    loop.run_until_complete(crMain(hotelUrls))
    